# Compare format

In [1]:
import numpy as np
import pandas as pd
import time
import _pickle as pkl
#import cPickle as pkl
import os
#import tables

batch_size = 10
embedding_size = 512
sample_num = 1000

a1 = np.random.normal(size=[embedding_size, sample_num])
label = np.random.normal(size=[embedding_size, 1])
all_batch = np.concatenate([a1, label], 1)
# 重新分割
# a1, a2 = np.split(all_batch, [-1], 1)

all_batches = []
for i in range(batch_size):
    all_batches.append(all_batch)
all_batches = np.array(all_batches)
print(all_batches.shape)


# dataframe
df = pd.DataFrame(all_batch)
df1 = pd.DataFrame()
for i in range(batch_size):
    df1 = pd.concat([df1,df])

(10, 512, 1001)


## 1. Pickle

In [2]:
s_t = time.time()
pkl_name = "a.pkl"
with open(pkl_name, "wb") as f:
    pkl.dump(all_batches, f)
pkl_in_time = time.time()-s_t
print("pkl dump costs {} sec".format(pkl_in_time))
s_t = time.time()
with open(pkl_name,'rb') as f:
    new_a = pkl.load(f)
pkl_out_time = time.time() - s_t
print("pkl load costs {} sec".format(pkl_out_time))
pkl_size = os.path.getsize(pkl_name)
print("pkl file size: {} byte, {} mb".format(pkl_size, float(pkl_size)/(1024*1024)))

pkl dump costs 0.057932376861572266 sec
pkl load costs 0.042009830474853516 sec
pkl file size: 41001122 byte, 39.10171699523926 mb


## 2. npy

In [3]:
s_t = time.time()
npy_name = "a.npy"
with open(npy_name, "wb") as f:
    np.save(f, arr=all_batches)
npy_in_time = time.time() - s_t
print("npy save costs {} sec".format(npy_in_time))
s_t = time.time()
with open(npy_name,'rb') as f:
    new_a = np.load(f)
npy_out_time = time.time() - s_t
print("npy load costs {} sec".format(npy_out_time))
npy_size = os.path.getsize(npy_name)
print("npy file size: {} byte, {} mb".format(npy_size, float(npy_size) / (1024*1024)))

npy save costs 0.03301215171813965 sec
npy load costs 0.02905440330505371 sec
npy file size: 41001088 byte, 39.1016845703125 mb


## 3. npz

In [4]:
s_t = time.time()
npz_name = "a.npz"
with open(npz_name, "wb") as f:
    np.savez(f, arr=all_batches)
npz_in_time = time.time() - s_t
print("npz save costs {} sec".format(npz_in_time))
s_t = time.time()
with open(npz_name,'rb') as f:
    npz_f= np.load(f)
    new_a = npz_f["arr"]
npz_out_time = time.time() - s_t
print("npz load costs {} sec".format(npz_out_time))
npz_size = os.path.getsize(npz_name)
print("npz file size: {} byte, {} mb".format(npz_size, float(npz_size) /(1024*1024)))

npz save costs 0.059032440185546875 sec
npz load costs 0.07599925994873047 sec
npz file size: 41001220 byte, 39.101810455322266 mb


## 4. h5

In [5]:
import tables

s_t = time.time()
table_name = "a.h5"
with tables.open_file(table_name,'w') as f:
#f = tables.open_file(table_name, 'w')
    atom = tables.Atom.from_dtype(all_batches.dtype)
    ds = f.create_carray(f.root, 'test_a', atom, all_batches.shape)
    ds[:] = all_batches
table_in_time = time.time() - s_t
print("table save costs {} sec".format(table_in_time))
s_t2 = time.time()
with tables.open_file(table_name,'r') as f:
#f2 = tables.openFile(table_name, "r")
    hdf5_data = f.root.test_a[:]
table_out_time = time.time() - s_t2
print("table load costs {} sec".format(table_out_time))
table_size = os.path.getsize(table_name)
print("table file size: {} byte, {} MB".format(table_size, float(table_size) / (1024*1024)))

ModuleNotFoundError: No module named 'packaging.version'

## 5. feather
feather安装：
pip install feather-format

In [ ]:
# !pip install feather-format -i https://pypi.mirrors.ustc.edu.cn/simple/

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2457 sha256=84f8278e2fbfb93560b530e6a0efee30736e6ba8d2a0070e2dcf0a109f23763b
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\b5\4d\cf\d3dd677e24ffa1538d30847a27bc8c5dcafb80676e82832c09
Successfully built feather-format


In [7]:
import feather

    #df1 = df1.append(pd.DataFrame(all_batch))
    #print(df1.shape)
s_t = time.time()
feather_name = 'a.feather'
with open(feather_name,'wb') as f:
    feather.write_dataframe(df1,f)
feather_in_time = time.time() -s_t
print("faether write costs {} sec".format(feather_in_time))
s_t2 = time.time()
with open(feather_name,'rb') as f:
    readFeather = feather.read_dataframe(f)
feather_out_time = time.time() -s_t2 

print("faether load costs {} sec".format(feather_out_time))
feather_size = os.path.getsize(feather_name)
print("feather file size: {} byte, {} MB".format(feather_size, float(feather_size) / (1024*1024)))

faether write costs 0.07100200653076172 sec
faether load costs 0.04601240158081055 sec
feather file size: 4650922 byte, 4.435464859008789 MB


## 6. Jay


In [8]:
# !pip install datatable -i https://pypi.mirrors.ustc.edu.cn/simple/

In [9]:
import datatable as dt

s_t = time.time()
jay_name = 'a.jay'
dt.Frame(df1).to_jay(jay_name)
jay_in_time = time.time() - s_t
print("jay write costs {} sec".format(jay_in_time))

s_t2= time.time()
jay_data =dt.fread(jay_name)
jay_out_time = time.time() - s_t2
print("jay read costs {} sec".format(jay_out_time))

jay_size = os.path.getsize(jay_name)
print("jay file size: {} byte, {} MB".format(jay_size, float(jay_size) / (1024*1024)))

jay write costs 0.41909265518188477 sec
jay read costs 0.024005651473999023 sec
jay file size: 41045088 byte, 39.143646240234375 MB


## 7. Parquet


In [10]:
#!pip install pyarrow -i https://pypi.mirrors.ustc.edu.cn/simple/

In [11]:
import pyarrow.parquet as pq
import pyarrow as pa
Parquet_name = 'a.parquet'

s_t = time.time()
table = pa.Table.from_pandas(df1)
pq.write_table(table,Parquet_name)
Parquet_in_time = time.time() - s_t
print("Parquet write costs {} sec".format(Parquet_in_time))

s_t2 = time.time()
parquet_data = pd.read_parquet(Parquet_name)
Parquet_out_time = time.time() - s_t2
print("Parquet read costs {} sec".format(Parquet_out_time))

parquet_size = os.path.getsize(Parquet_name)
print("Parquet file size: {} byte, {} MB".format(parquet_size, float(parquet_size) / (1024*1024)))

Parquet write costs 0.2930636405944824 sec
Parquet read costs 0.19948840141296387 sec
Parquet file size: 5662560 byte, 5.400238037109375 MB
